# Dask Flood Mapper

In [ ]:
from dask_flood_mapper import flood
from bokeh.models import FixedTicker
import hvplot.xarray

In [ ]:
import dask
dask.config.set(temporary_directory='/tmp')
from dask.distributed import Client, wait
client = Client(processes=False, threads_per_worker=2,
                n_workers=1, memory_limit='20GB')

/home/mschobbe/.local/share/virtualenvs/dask-flood-mapper-z0eq0g_n/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45961 instead
  warnings.warn(


/home/mschobbe/.local/share/virtualenvs/dask-flood-mapper-z0eq0g_n/lib/python3.12/site-packages/dask/_task_spec.py:740: RuntimeWarning: All-NaN slice encountered
  return self.func(*new_argspec, **kwargs)
/home/mschobbe/.local/share/virtualenvs/dask-flood-mapper-z0eq0g_n/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: divide by zero encountered in divide
  return self.func(*new_argspec)


In [3]:
time_range = "2022-10-11/2022-10-25"
bbox = [12.3, 54.3, 13.1, 54.6]
fd = flood.decision(bbox=bbox, datetime=time_range).compute()
fd

sig0_dc processed
hpar_dc processed
plia_dc processed


<xarray.DataArray 'decision' (time: 8, y: 1048, x: 2793)> Size: 187MB
array([[[nan, nan, nan, ...,  0., nan, nan],
        [ 0.,  0.,  0., ...,  0., nan, nan],
        [ 0.,  0.,  0., ...,  0., nan, nan],
        ...,
        [nan, nan,  0., ...,  0.,  0.,  0.],
        [nan, nan,  0., ...,  0., nan, nan],
        [nan, nan,  0., ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan,  0., ...,  0.,  0.,  0.],
        [nan, nan,  0., ...,  0., nan, nan],
        [nan, nan,  0., ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan,  0., ...,  0.,  0.,  0.],
        [nan, nan,  0., ...,  0., nan, nan],
        [nan, nan,  0., ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ...,  0., nan, nan],
        [ 0.,  0.,  0., ...,  0., nan, nan],
        [ 0.,  0.,  0., ...,  0., nan, nan],
        ...,
        [nan, nan,  0., ...,  0.,  0.,  0.],
        [nan, nan,  0., ...,  0., nan, nan],
        [nan, nan,  0., ..., nan, nan, nan]]])
Coordinates:
  * x            (x) float64 22kB 12.3 12.3 12.3 12.3 ... 13.1 13.1 13.1 13.1
  * y            (y) float64 8kB 54.6 54.6 54.6 54.6 ... 54.3 54.3 54.3 54.3
  * time         (time) datetime64[ns] 64B 2022-10-11T05:25:01 ... 2022-10-23...
    spatial_ref  int64 8B 0
Attributes:
    _FillValue:  nan

In [3]:
from importlib.resources import files
import xarray as xr
data_text = files('dask_flood_mapper.data').joinpath('wcover.tif')
wcover = xr.open_dataarray(data_text)
wcover

<xarray.DataArray 'band_data' (band: 1, y: 1048, x: 2793)> Size: 23MB
[2927064 values with dtype=float64]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 22kB 12.3 12.3 12.3 12.3 ... 13.1 13.1 13.1 13.1
  * y            (y) float64 8kB 54.6 54.6 54.6 54.6 ... 54.3 54.3 54.3 54.3
    spatial_ref  int64 8B ...
Attributes:
    AREA_OR_POINT:  Area
    long_name:      wcover

In [8]:
fd = fd.where(wcover != 80)
fd

<xarray.DataArray 'decision' (time: 8, y: 857, x: 1716, band: 1)> Size: 94MB
array([[[[nan],
         [nan],
         [nan],
         ...,
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         ...,
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         ...,
...
         ...,
         [ 0.],
         [ 0.],
         [ 0.]],

        [[nan],
         [nan],
         [nan],
         ...,
         [ 0.],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         ...,
         [nan],
         [nan],
         [nan]]]])
Coordinates:
  * x            (x) float64 14kB 12.3 12.3 12.3 12.3 ... 13.1 13.1 13.1 13.1
  * y            (y) float64 7kB 54.6 54.6 54.6 54.6 ... 54.3 54.3 54.3 54.3
  * time         (time) datetime64[ns] 64B 2022-10-11T05:25:01 ... 2022-10-23...
    spatial_ref  int64 8B 0
  * band         (band) int64 8B 1
Attributes:
    _FillValue:  nan

In [9]:
colorbar_opts = {
    "major_label_overrides": {
        0: "non-flood",
        1: "flood",
    },
    "ticker": FixedTicker(ticks=[0, 1]),
}
fd.hvplot.image(
    x="x", y="y", rasterize=True, geo=True, cmap=["rgba(0, 0, 1, 0.1)", "darkred"]
).opts(frame_height=400, colorbar_opts={**colorbar_opts})

/home/mschobbe/.local/share/virtualenvs/dask-flood-mapper-z0eq0g_n/lib/python3.12/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


BokehModel(combine_events=True, render_bundle={'docs_json': {'2b6a27e7-3058-4c07-8d2e-e7f35d0f5927': {'version…

In [10]:
client.close()